In [1]:
pip install pandas

     |████████████████████████████████| 9.7 MB 89.0 MB/s 
     |████████████████████████████████| 15.4 MB 11.1 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.
